In [2]:
%pylab nbagg
from tvb.simulator.lab import *
from sklearn.ensemble import ExtraTreesRegressor
import operator
from scipy.signal import butter, lfilter
import scipy
import sim
import pickle
import copy
import os.path

Populating the interactive namespace from numpy and matplotlib
   INFO  log level set to INFO


In [3]:
result_no = 0

In [4]:
path = 'Graphs/Pineau-Results/'

#Function Definitions

In [5]:
def action_params(action):
    if action == 0:
        t = 0
        p = 0
        a = 0
    elif action == 1:
        t = 200
        p = 1
        a = 1
    elif action == 2:
        t = 67
        p = 1
        a = 1
    elif action == 3:
        t = 200
        p = 1
        a = 1.25
    elif action == 4:
        t = 67
        p = 1
        a = 1.25
    elif action == 5:
        t = 200
        p = 1
        a = 1.5
    elif action == 6:
        t = 67
        p = 1
        a = 1.5
    elif action == 7:
        t = 200
        p = 1
        a = 1.75
    elif action == 8:
        t = 67
        p = 1
        a = 1.75
    elif action == 9:
        t = 200
        p = 1
        a = 2
    elif action == 10:
        t = 67
        p = 1
        a = 2
    elif action == 11:
        t = 200
        p = 1
        a = 2.25
    elif action == 12:
        t = 67
        p = 1
        a = 2.25
    elif action == 13:
        t = 200
        p = 1
        a = 2.5
    elif action == 14:
        t = 67
        p = 1
        a = 2.5
    elif action == 15:
        t = 100
        p = 1
        a = 1
    elif action == 16:
        t = 100
        p = 1
        a = 1.25
    elif action == 17:
        t = 100
        p = 1
        a = 1.5
    elif action == 18:
        t = 100
        p = 1
        a = 1.75
    elif action == 19:
        t = 100
        p = 1
        a = 2
    elif action == 20:
        t = 100
        p = 1
        a = 2.25
    elif action == 21:
        t = 100
        p = 1
        a = 2.5
    elif action == 22:
        t = 20
        p = 1
        a = 0.5
    elif action == 23:
        t = 20
        p = 1
        a = 0.75
    elif action == 24:
        t = 20
        p = 1
        a = 1
    elif action == 25:
        t = 10
        p = 1
        a = 0.5
    elif action == 26:
        t = 10
        p = 1
        a = 0.75
    elif action == 27:
        t = 10
        p = 1
        a = 1
     
     
        
    return (t,p,a)


#Import Test Data

In [6]:
parameters = [
             # [1,200,1,0,'test',1,1000,200],
              [0, 0, 0, 0, 'Control-0Hz', 0, 20000, 200],
              [1, 200, 1, 0, '5KHz-1pw-1amp',1,20000, 200],
              [1, 67, 1, 0, '15KHz-1pw-1amp',2,20000, 200],
              [1.25, 200, 1, 0, '5KHz-1pw-1_25amp',3,20000, 200],
              [1.25, 67, 1, 0, '15KHz-1pw-1_25amp',4,20000,200],
              [1.5, 200, 1, 0, '5KHz-1pw-1_5amp',5,20000, 200],
              [1.5, 67, 1, 0, '15KHz-1pw-1_5amp',6,20000, 200],
              [1.75, 200, 1, 0, '5KHz-1pw-1_75amp',7,20000, 200],
              [1.75, 67, 1, 0, '15KHz-1pw-1_75amp',8,20000,200],
              [2, 200, 1, 0, '5KHz-1pw-2amp', 9, 20000, 200],
              [2, 67, 1, 0, '15Khz-1pw-2amp', 10, 20000, 200],
              [2.25, 200, 1, 0, '5KHz-1pw-2_25amp', 11, 20000, 200],
              [2.25, 67, 1, 0, '15KHz-1pw-2_25amp', 12, 20000, 200],
              [2.5, 200, 1, 0, '5KHz-1pw-2_5amp', 13, 20000, 200],
              [2.5, 67, 1, 0, '15KHz-1pw-2_5amp', 14, 20000, 200],
            
              [1, 100, 1, 0, '10KHz-1pw-1amp',15,20000, 200],
              [1.25, 100, 1, 0, '10KHz-1pw-1_25amp',16,20000, 200],
              [1.5, 100, 1, 0, '10KHz-1pw-1_5amp',17,20000, 200],
              [1.75, 100, 1, 0, '10KHz-1pw-1_75amp',18,20000,200],
              [2, 100, 1, 0, '10KHz-1pw-2amp',19,20000,200],
              [2.25, 100, 1, 0, '10KHz-1pw-2_25amp',20,20000,200],
              [2.5, 100, 1, 0, '10KHz-1pw-2_5amp',21,20000,200],
    
              [0.5, 20, 1, 0, '50KHz-1pw-0_5amp',22,20000, 200],
              [0.75, 20, 1, 0, '50KHz-1pw-0_75amp',23,20000, 200],
              [1, 20, 1, 0, '50KHz-1pw-1amp',24,20000, 200],
    
              [0.5, 10, 1, 0, '100KHz-1pw-0_5amp',25,20000, 200],
              [0.75, 10, 1, 0, '100KHz-1pw-0_75amp',26,20000, 200],
              [1, 10, 1, 0, '100KHz-1pw-1amp',27,20000, 200],   
            ]
datapath = "Graphs/bigdata/"
results = []
for f in parameters:
    filepath = datapath + f[4] + '.p'
    if os.path.isfile(filepath):
        readfile = open(filepath, 'r')
        in_var = pickle.load(readfile)
        results.append([in_var[0],in_var[1],in_var[2]])

#Analyze test set
Use 800 ms frames that move by 200 ms (dt = 200, frame_len = 800)
ictal_log has the same format as action_log, where 1 corresponds to ictal and 0 corresponds to inter-ictal states.

In [7]:
def analyze_frame(d, dt, frame_len):
    data = np.array(d)
    data_sq = square(data)
    data_fft = abs(fft.fft(data))**2/10000
    
    
    #Full
    maxi_long = max(data)
    mini_long = min(data)
    rng_long = maxi_long-mini_long
    avg_long = sum(data)/len(data)
    tot_eng_long = sum(data_fft)
    loF_eng_long = sum(data_fft[:len(data_fft)/3])
    miF_eng_long = sum(data_fft[len(data_fft)/3:2*len(data_fft)/3])
    hiF_eng_long = sum(data_fft[2*len(data_fft)/3:])

    #Half 
    half = len(data)/2
    data_fft_half = abs(fft.fft(data[half:]))
    maxi_half = max(data[half:])
    mini_half = min(data[half:])
    rng_half = maxi_half-mini_half
    avg_half = sum(data[half:])/len(data[half:])
    tot_eng_half = sum(data_fft_half)
    loF_eng_half = sum(data_fft_half[:len(data_fft_half)/3])
    miF_eng_half = sum(data_fft_half[len(data_fft_half)/3:2*len(data_fft_half)/3])
    hiF_eng_half = sum(data_fft_half[2*len(data_fft_half)/3:])


    #Quarter
    qt = 3*len(data)/4
    data_fft_qt = abs(fft.fft(data[qt:]))
    maxi_qt = max(data[qt:])
    mini_qt = min(data[qt:])
    rng_qt = maxi_qt=mini_qt
    avg_qt = sum(data[qt:])/len(data[qt:])
    tot_eng_qt = sum(data_fft_qt)
    loF_eng_qt = sum(data_fft_qt[:len(data_fft_qt)/3])
    miF_eng_qt = sum(data_fft_qt[len(data_fft_qt)/3:2*len(data_fft_qt)/3])
    hiF_eng_qt = sum(data_fft_qt[2*len(data_fft_qt)/3:])


    #Eighth 
    eit = 7*len(data)/8
    data_fft_eit = abs(fft.fft(data[eit:]))
    maxi_eit = max(data[eit:])
    mini_eit = min(data[eit:])
    rng_eit = maxi_eit-mini_eit
    avg_eit = sum(data[eit:])/len(data[eit:])
    tot_eng_eit = sum(data_fft_eit)
    loF_eng_eit = sum(data_fft_eit[:len(data_fft_eit)/3])
    miF_eng_eit = sum(data_fft_eit[len(data_fft_eit)/3:2*len(data_fft_eit)/3])
    hiF_eng_eit = sum(data_fft_eit[2*len(data_fft_eit)/3:])


    state = [
             #stim_T, stim_pw, stim_amp,
             #maxi_long, mini_long, rng_long, avg_long, tot_eng_long, loF_eng_long, miF_eng_long, hiF_eng_long,
             #maxi_half, mini_half, rng_half, avg_half, tot_eng_half, loF_eng_half, miF_eng_half, hiF_eng_half,
             maxi_qt, mini_qt, rng_qt, avg_qt, tot_eng_qt, loF_eng_qt, miF_eng_qt, hiF_eng_qt, 
             maxi_eit, mini_eit, rng_eit, avg_eit, tot_eng_eit, loF_eng_eit, miF_eng_eit, hiF_eng_eit, 
            ]
    return state

In [28]:
def analyze_set(t, d, action_log, ictal_log, dt, time):
    frame_len = 800
    
    
    n_frames = (time-frame_len)/dt
    
    state_transition = []
    
    ## HAND ANNOTATE
    ictal_log = []
    ##
    
    for i in range((n_frames)):
    
        time = t[i*dt:i*dt+frame_len]
        data = np.array(d[i*dt:i*dt+frame_len,0,9,0])
        #ictal = ictal_log[i + frame_len/dt]
        action = action_log[i+frame_len/dt]
        data_sq = square(data)
        data_fft = abs(fft.fft(data))**2/10000
        
        ##HAND ANNOTATE
        if sum(data[3*len(data)/4:])/len(data[3*len(data)/4:])/len(data[3*len(data)/4:]) < 0:
            ictal = 1
        else: ictal = 0
        ictal_log.append(ictal)
        ##HAND ANNOTATE
        
        
        #Stim
        action_p = action_params(action)
        stim_T = action_p[0]
        stim_pw = action_p[1]
        stim_amp = action_p[2]
        
        #Use analyze_frame instead of this block
        
        #Full
        maxi_long = max(data)
        mini_long = min(data)
        rng_long = maxi_long-mini_long
        avg_long = sum(data)/len(data)
        tot_eng_long = sum(data_fft)
        loF_eng_long = sum(data_fft[:len(data_fft)/3])
        miF_eng_long = sum(data_fft[len(data_fft)/3:2*len(data_fft)/3])
        hiF_eng_long = sum(data_fft[2*len(data_fft)/3:])
        
        #Half 
        half = len(data)/2
        data_fft_half = abs(fft.fft(data[half:]))
        maxi_half = max(data[half:])
        mini_half = min(data[half:])
        rng_half = maxi_half-mini_half
        avg_half = sum(data[half:])/len(data[half:])
        tot_eng_half = sum(data_fft_half)
        loF_eng_half = sum(data_fft_half[:len(data_fft_half)/3])
        miF_eng_half = sum(data_fft_half[len(data_fft_half)/3:2*len(data_fft_half)/3])
        hiF_eng_half = sum(data_fft_half[2*len(data_fft_half)/3:])
        
        
        #Quarter
        qt = 3*len(data)/4
        data_fft_qt = abs(fft.fft(data[qt:]))
        maxi_qt = max(data[qt:])
        mini_qt = min(data[qt:])
        rng_qt = maxi_qt=mini_qt
        avg_qt = sum(data[qt:])/len(data[qt:])
        tot_eng_qt = sum(data_fft_qt)
        loF_eng_qt = sum(data_fft_qt[:len(data_fft_qt)/3])
        miF_eng_qt = sum(data_fft_qt[len(data_fft_qt)/3:2*len(data_fft_qt)/3])
        hiF_eng_qt = sum(data_fft_qt[2*len(data_fft_qt)/3:])
        
        
        #Eighth 
        eit = 7*len(data)/8
        data_fft_eit = abs(fft.fft(data[eit:]))
        maxi_eit = max(data[eit:])
        mini_eit = min(data[eit:])
        rng_eit = maxi_eit-mini_eit
        avg_eit = sum(data[eit:])/len(data[eit:])
        tot_eng_eit = sum(data_fft_eit)
        loF_eng_eit = sum(data_fft_eit[:len(data_fft_eit)/3])
        miF_eng_eit = sum(data_fft_eit[len(data_fft_eit)/3:2*len(data_fft_eit)/3])
        hiF_eng_eit = sum(data_fft_eit[2*len(data_fft_eit)/3:])
        
        
        state = [
                 #stim_T, stim_pw, stim_amp,
                 #maxi_long, mini_long, rng_long, avg_long, tot_eng_long, loF_eng_long, miF_eng_long, hiF_eng_long,
                 #maxi_half, mini_half, rng_half, avg_half, tot_eng_half, loF_eng_half, miF_eng_half, hiF_eng_half,
                 maxi_qt, mini_qt, rng_qt, avg_qt, tot_eng_qt, loF_eng_qt, miF_eng_qt, hiF_eng_qt, 
                 maxi_eit, mini_eit, rng_eit, avg_eit, tot_eng_eit, loF_eng_eit, miF_eng_eit, hiF_eng_eit, 
                ]
        
        action_penalty = 0
        if action != 0:
            action_penalty = -1
        action_weight = 0.8
        
      

        reward = - ictal + action_weight * action_penalty
        
        #use previous state in learning
        state_transition.append([state, action, reward])
        
    return state_transition

#Set up dataset

In [32]:
all_state_transitions = []
for i in range(1,len(results)):
    t = results[i][0]
    d = results[i][1]
    a = results[i][2]
    i = []
    dt = 200
    ti = 20000
    
    s_t = analyze_set(t, d, a, i, dt, ti)
    all_state_transitions.append(s_t)

#Fitted Q Iteration
Just like Guez did it

In [33]:
i = 0
input_tree = []
output_tree = []
nextstate = []
for state_transition in all_state_transitions:
    for i in range(len(state_transition) - 1):
        if i > 0:
            previous_state = copy.deepcopy(state_transition[i-1][0])
            result_state = copy.deepcopy(state_transition[i][0])
            action_taken = state_transition[i][1]
            reward = state_transition[i][2]
            relative_reward = reward - state_transition[i-1][2]
            
            state_action = append(previous_state,action_taken)#[previous_state, action] #should they be in a single array?
            out = reward #relative reward or reward!?
            
            input_tree.append(state_action)
            output_tree.append(out)
            nextstate.append(result_state)

In [31]:
results[0][2][0]

0

In [34]:
def iterate_q(ins, outs, nextstate, iterations, valid_action):
    etrees = ExtraTreesRegressor()
    N = 0;
    for x in range(iterations):
        N += 1
        out_new = copy.copy(outs)
        if N > 1:
            for i in range(len(outs)):
                bestaction = random.choice(valid_action)
                
                bestval = etrees.predict(append(nextstate[i], bestaction).reshape(1,-1))
                
                for action in valid_action:
                    val = etrees.predict(append(nextstate[i], action).reshape(1,-1))
                    if val > bestval:
                        bestaction = action
                        bestval = val
                #print ("nextstate: " + str(nextstates[i]) + " best action: " + str(bestaction) + " val: "+str(bestval) )
                out_new[i] += 0.9*bestval #gamma = 0.9
        etrees.fit(ins, np.ravel(out_new))
        #if x%(iterations/10.) == 0:
        print "iteration " + str(x)
        
        
    return etrees

In [35]:
predictor = iterate_q(input_tree, output_tree, nextstate,10,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


#Test the decision policy
And really really hope it works.


In [36]:
epileptic = [9]
almost_epileptic = []
stimulus_target = [9]
stimulus_amplitude = 0
stimulus_weight = 2 #0-2 for each region
onset = 0
period = 0
pulse_width = 0
sim_time = 10000
dt = 200
frame_length = 800
ic_buffer = 3000
(s, stimulus) = sim.set_sim(epileptic, almost_epileptic, stimulus_target, stimulus_amplitude, 
              stimulus_weight, onset, period, pulse_width, sim_time)


(t_t, t_d), (aa, aa1), (bb, bb1) = s.run(simulation_length = ic_buffer)


valid_action = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
test_states = []
action_log = []

WARNING  File 'hemispheres' not found in ZIP.


In [37]:
#(s_control, stim_control) = sim.set_sim(epileptic, almost_epileptic, stimulus_target, stimulus_amplitude, 
#              stimulus_weight, onset, period, pulse_width, sim_time)
#(t_control, d_control), (x, y), (z,w) = s_control.run(simulation_length = ic_buffer)
#(t_c, d_c), (xxxxx, yyyyy), (xxxx, yyyy) = s_control.run(simulation_length = sim_time)
#t_contol = np.append(t_contol, t_c, axis = 0)
#d_control = np.append(d_control, d_c, axis = 0)
#who needs a control group?

In [38]:
t_all = copy.copy(t_t)
d_all = copy.copy(t_d)

In [39]:
for i in range(sim_time/dt):
    if i%5 == 0:
        print "time: " + str(i*dt)
    if i == 0:
        action_log.append(0)
        
    (t,d), (aaa, bbb), (ccc,ddd) = s.run(simulation_length = dt)
    t_all = np.append(t_all, t, axis = 0)
    d_all = np.append(d_all, d, axis = 0)
    
    frame_data = d_all[ ic_buffer + (i+1)*dt - frame_length : ic_buffer + (i+1)*dt ,0,9,0] 
    frame_time = t_all[ ic_buffer + (i+1)*dt - frame_length : ic_buffer + (i+1)*dt ] 
    
    state = analyze_frame(frame_data, dt, frame_length)
    
    bestaction = 5
    bestval = predictor.predict(append(state, bestaction).reshape(1,-1))[0]
    
    for action in valid_action:
        val = predictor.predict(append(state, action).reshape(1,-1))[0]
        #print("     Action: " + str(action) + " Value: " + str(val))
        if val > bestval:
            bestaction = action
            bestval = val
    #print "best value is : " + str(bestval) + "\n"
    #print "best action is : " + str(bestaction) + "\n"
    action = bestaction
    
    
    action_p = action_params(action)
    T = action_p[0]
    tau = action_p[1]
    amp = action_p[2]
    
    stimulus.temporal.parameters["amp"] = amp
    stimulus.temporal.parameters["T"] = T
    stimulus.temporal.parameters["tau"] = tau
    
    test_states.append(state)
    action_log.append(action)

time: 0
time: 1000
time: 2000
time: 3000
time: 4000
time: 5000
time: 6000
time: 7000
time: 8000
time: 9000


In [40]:
plot_save_test(t_all, d_all, action_log,test_states,sim_time,dt,"10iterations", 5)

In [19]:
def plot_save_test(t_all, d_all, action_log, test_states, sim_time, dt, filename, result_no):
    fig, ax1 = pyplot.subplots(figsize=(15,10))
    ax2 = ax1.twinx()
    ioff()
    ax1.plot(t_all[3000:]-2999.5,d_all[3000:,0,9,0], 'r', label = "Local Field Potential")

    #ax2.plot((t_all[3000:3051]-2999.5)*200, action_log)
    for i in range(sim_time/dt):
        if i == 0:
            timeframe = ax1.plot([i*dt, i*dt], [-3, 3],'g', alpha = 0.1, label = "Frame Start")
        else:
            ax1.plot([i*dt, i*dt], [-3, 3],'g', alpha = 0.1)
    i = 0
    framestart = 0
    stim = 0
    for n in range(len(t_all[3000:])):
        if n % dt == 0 :

            i += 1
            framestart = n
            p = action_params(action_log[i-1])
            period = p[0]
            amp = p[2]
        if  period != 0 and (n - framestart == 0 or (n - framestart) % period ==0 ) :

            if stim == 0:
                ax2.plot([t_all[3000+n]-2999.5, t_all[3000+n]-2999.5], [-amp, amp], 'b', linewidth = 1.5, alpha = 0.5, label = "Stimulus")
                stim += 1
            else:
                ax2.plot([t_all[3000+n]-2999.5, t_all[3000+n]-2999.5], [-amp, amp], 'b', linewidth = 1.5, alpha = 0.5)
    title("Neurostimulation with Offline Reinforcement Learning \n" + filename)


    red = matplotlib.patches.Patch(color='red', label = 'LFP')
    blue = matplotlib.patches.Patch(color='blue', label = 'Stimulus')
    green = matplotlib.patches.Patch(color='green',alpha=0.3, label = 'Frame Start')

    legend(handles=[red,blue,green])


    #show()

    
    fig.savefig(path + "result-" + filename + str(result_no))
    savefile = open(path + "result-" + filename + str(result_no)+'.p','w')
    pickle.dump((t_all, d_all, action_log, test_states), savefile)

In [39]:
action_log

[0,
 14,
 0,
 5,
 5,
 5,
 9,
 14,
 10,
 10,
 0,
 13,
 11,
 11,
 5,
 5,
 11,
 8,
 12,
 5,
 10,
 10,
 5,
 5,
 0,
 11,
 11,
 8,
 13,
 14,
 10,
 13,
 13,
 13,
 5,
 5,
 7,
 0,
 11,
 10,
 10,
 0,
 5,
 9,
 13,
 9,
 5,
 6,
 5,
 5,
 5]